# Exploring spatial and biodiversity data

```{admonition} Summary
:class: hint
This walkthrough provides a complete data processing example from start to finish.

The example is based on two data sources:
- Biodiversity data from [Lebendiger Atlas der Natur Deutschlands(LAND)](https://land.gbif.de/): Species observations as coordinate-based vector data.
- Spatial data from [IOER Monitor](https://www.ioer-monitor.de/): Aggregated and processed land use data that is provided in raster format (WCS).

In this section, we will retrieve data from both sources and then use intersection and aggregation methods to create an integrated visualisation. While Python offers multiple ways to process data, we will focus on a single approach without exploring alternative routes. Where further details are useful, cross-references to the reference documentation (Part III) or external sources are provided.
```

## Lebendiger Atlas der Natur Deutschlands (LAND) – How and where to find the data?

The LAND species observations data are available at [LAND - Lebendiger Atlas der Natur Deutschlands](https://land.gbif.de/). LAND compiles species observations in Germany, making them accessible for research and the public. It displays selected occurrence datasets for Germany that are publicly available via the [Global Biodiversity Information Facility (GBIF)](Gbif.org). This connection allows us to explore biodiversity data both through [LAND.gbif.de](https://land.gbif.de/) and directly via GBIF’s global database.

There are two ways to download data:
- **Via web browser**: Search and download individual data files directly from [LAND](https://land.gbif.de/).
- **Via code**: Use the [GBIF Species Application Programming Interface (API)](https://techdocs.gbif.org/en/openapi/v1/species) access data programmatically.

For both methods, and especially for larger downloads, registration is required. However, we can explore a small example query using the Species API without needing a registration or key.

### Example: Finding the scientific name of a species

Let’s say we want to find the scientific name of a species but only have its common name — for example, the `English Sparrow` - and want to find its scientific name. The [GBIF.org species search](https://www.gbif.org/species/search?q=english%20sparrow) provides the correct answer.

Since [GBIF.org](https://www.gbif.org) we can likely access the same data and information programmatically. Let's explore how!

```{figure} https://images.naturalis.nl/original/csr%2FD80_8153.jpg
:name: sparrow-graphic

The English Sparrow (GBIF.org).
```

### GBIF Species API

```{dropdown} How do I know how to work with APIs?
Good APIs have documentation. The [GBIF Species API documentation](https://techdocs.gbif.org/en/openapi/v1/species#/Searching%20names/searchNames) even allows to try the API directly in a web browser. Go to <a href="https://techdocs.gbif.org/en/openapi/v1/species#/Searching%20names/searchNames">https://techdocs.gbif.org/en/openapi/v1/species#/Searching%20names/searchNames</a> and use the general species search for "English Sparrow". Simply enter `English Sparrow` in the field labeled `q (query)`(Simple full text search parameter).
```

- Retrieving data from APIs requires a specific syntax that is different for each service.
- commonly, there is an **endpoint** (a url) that returns data in a structured format (e.g. **json**, see [Part III: Common spatial file formats](302_file_formats))
- most APIs require you to authenticate, but not all (e.g. Instagram, commons.wikimedia.org)

We can do the same in Python. Load gbif search answer for a specific common name search. Get the scientific names matching "English Sparrow".

In [1]:
search_name = "English Sparrow"
query_url = f'https://api.gbif.org/v1/species/search?q={search_name}'

(content:references:admonition)=
```{admonition} Use your own common name!
:class: dropdown, attention
Optionally replace "English Sparrow" with another location above
```

```{admonition} Syntax: f'{}' ?
:class: tip, dropdown
This is called an f-string, [a convenient python convention](https://realpython.com/python-f-strings/) to concat strings and variables.
```

Get the json-answer from the API (without login!).

In [2]:
import requests

json_text = None
response = requests.get(url=query_url)

The data can be found in `response.text` (in case the API answered).

In [3]:
import json
json_data = json.loads(response.text)

We can see that the correct answer, `Passer domesticus`, is hidden in the nested json structure:
(content:references:nub-id)=

In [4]:
print(json.dumps(json_data, indent=2)[0:550])

{
  "offset": 0,
  "limit": 20,
  "endOfRecords": true,
  "count": 5,
  "results": [
    {
      "key": 100220560,
      "datasetKey": "b351a324-77c4-41c9-a909-f30f77268bc4",
      "nubKey": 5231190,
      "scientificName": "Passer domesticus",
      "canonicalName": "Passer domesticus",
      "nameType": "SCIENTIFIC",
      "taxonomicStatus": "ACCEPTED",
      "origin": "SOURCE",
      "numDescendants": 0,
      "numOccurrences": 0,
      "taxonID": "420",
      "habitats": [],
      "nomenclaturalStatus": [],
      "threatStatuses": [],
     


We can access the most relevant first result and print this name directly. Below, we walk the json path from the `"results"`, which is a list, access the first entry by using `[0]` and the access its key of the name `scientificName`.

In [5]:
json_data["results"][0]["scientificName"]

'Passer domesticus'

We can also see the taxon id, a unique reference within the GBIF database that we will need in the next chapter to query occurrences:
(content:references:passer-id)=

In [6]:
json_data["results"][0]["taxonID"]

'420'

`````{admonition} Continue in the next section..
:class: seealso
Passer domesticus will be our example species for which we will retrieve observations [in the next section](202_data_retrieval).
`````

## IÖR-Data – How and where to find the data?


The IÖR-Monitor Data is available at [https://monitor.ioer.de/](https://monitor.ioer.de/). All IÖR-Monitor data is available as Web Feature Service (WFS) and Web Coverage Service (WCS).
You can see a full list of services [here](https://monitor.ioer.de/monitor_api/services).

```{figure} https://www.ioer-monitor.de/fileadmin/user_upload/monitor/img/Ergebnisse/siedlungsdichte.png
:name: monitor-graphic

Wie dicht leben wir? [IÖR Monitor data](https://www.ioer-monitor.de/ergebnisse/analyseergebnisse/wie-dicht-leben-wir/).
```

```{admonition} Registration necessary
:class: warning
To retrieve data from the IÖR-Monitor API, [registration is necessary](https://www.ioer-monitor.de/login/).

In order to follow the steps below, you need to:
1. Register [monitor.ioer.de/monitor_api/signup](https://monitor.ioer.de/monitor_api/signup),
2. Create an API key,
3. Store this API key in a file called `.env` as `API_KEY=xyz`
```

```{admonition} Continue in the next section..
:class: seealso
[See the next section](202_data_retrieval) where we explain how to access the IÖR-WCS raster data for green cover in Saxony.
```

## Example research question

Now we have a research question. In the following example workflow, we investigate whether there is a positive correlation between sparrow density and settlement areas. Since sparrows are known to prefer settled areas, this should be visible with the IÖR monitor and the NFDI4Biodiversity data.

Furthermore, we want to see if there is a population trend for the sparrow in Saxony. {cite:ts}`salek_house_2015` observed an overall decreasing population trend for the sparrow.

> _Populations of House and Tree Sparrows have rapidly declined in various breeding habitats throughout their European distribution range; however, the strongest decline was recorded within urban environments._
{cite:p}`salek_house_2015`

Is this also true for Saxony? [Let's see..](202_data_retrieval)

## References

```{bibliography}
:style: unsrt
:filter: docname in docnames
```